# Data analysis

In [8]:
import pandas as pd
import os
import re # For regular expressions
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, f_oneway
import colorsys
import matplotlib.colors as mcolors
import numpy as np

# --- Configuration ---
BASE_DIR_3D = '/home/kirill/Desktop/For_Kirill/Iba1_Morpho_BB_Blind (Copy)'
GROUP_INFO_FILE_3D = '/home/kirill/Desktop/For_Kirill/iba1 Morpho_group.csv'
METRICS_FILENAME_3D = 'metrics_df_ramified.csv'

BASE_DIR_2D = '/home/kirill/Desktop/For_Kirill/microglial distance_2D_tiff (Copy)'
METRICS_FILENAME_2D = 'metrics_df_ramified_2d.csv'
# No explicit group file for 2D, we'll derive or assign a default group.

EXPORT_BASE_DIRECTORY = '/home/kirill/Desktop/For_Kirill/Takeshi_analysis_combined'

# Create export directory if it doesn't exist
if not os.path.exists(EXPORT_BASE_DIRECTORY):
    os.makedirs(EXPORT_BASE_DIRECTORY)
    print(f"Created base export directory: {EXPORT_BASE_DIRECTORY}")

ORIGINAL_CUSTOM_PALETTE = ['#00312F', '#1D2D46', '#46000D', '#5F3920', '#573844', '#424313', '#7A7A30', '#307A7A'] # Added more for potential new groups
BOXPLOT_WIDTH = 0.5
BOXPLOT_LINEWIDTH = 2.0
all_stats_records = []

# --- Helper Functions (make_pastel, format_axis_label, get_significance_asterisks, format_p_value_for_display) ---
# (These remain the same as in the previous version, so I'll omit them for brevity here but they should be included)
def make_pastel(hex_color, lightness_scale=0.7, saturation_scale=0.6):
    try:
        rgb_normalized = mcolors.to_rgb(hex_color)
        h, l, s = colorsys.rgb_to_hls(rgb_normalized[0], rgb_normalized[1], rgb_normalized[2])
        l_pastel = l + (1.0 - l) * lightness_scale; l_pastel = min(1.0, max(0.0, l_pastel))
        s_pastel = s * saturation_scale; s_pastel = min(1.0, max(0.0, s_pastel))
        rgb_pastel_normalized = colorsys.hls_to_rgb(h, l_pastel, s_pastel)
        return mcolors.to_hex(rgb_pastel_normalized)
    except ValueError: return hex_color

def format_axis_label(label_text):
    parts = label_text.split('_'); formatted_parts = []
    for part in parts:
        if part.lower() == "um": formatted_parts.append("µm" + part[2:])
        elif part.lower() == "um2": formatted_parts.append("µm²")
        elif part.lower() == "um3": formatted_parts.append("µm³")
        else: formatted_parts.append(part.capitalize())
    return " ".join(formatted_parts)

def get_significance_asterisks(p_value):
    if p_value is None: return ""
    if p_value < 0.001: return "***"
    if p_value < 0.01: return "**"
    if p_value < 0.05: return "*"
    return "n.s."

def format_p_value_for_display(p_value):
    if p_value is None: return ""
    sig = get_significance_asterisks(p_value)
    if sig == "n.s.": return f"n.s. (p={p_value:.3f})"
    if p_value < 0.001: return f"p < 0.001{sig}"
    return f"p={p_value:.3f}{sig}"

PASTEL_PALETTE = [make_pastel(color) for color in ORIGINAL_CUSTOM_PALETTE]
# --- END Helper Functions ---

# --- Data Loading and Processing Function ---
def load_and_process_dataset(dataset_name_tag, base_dir, metrics_filename, 
                             image_dir_pattern_func, # Function to check if a dir is an image dir
                             group_assignment_df_external=None, 
                             derive_group_from_image_id_func=None):
    print(f"\n--- Loading Dataset: {dataset_name_tag} ---")
    all_metrics_data = []
    if not os.path.isdir(base_dir):
        print(f"ERROR: Base directory '{base_dir}' for {dataset_name_tag} not found.")
        return pd.DataFrame()

    try:
        # List all items in base_dir, then filter to get potential image_id_dirs
        potential_dirs = os.listdir(base_dir)
        image_id_dirs = sorted([d for d in potential_dirs if image_dir_pattern_func(d, base_dir)])
    except Exception as e:
        print(f"ERROR: Could not list or filter directories in '{base_dir}': {e}")
        return pd.DataFrame()
        
    print(f"Found {len(image_id_dirs)} potential image directories for {dataset_name_tag}.")

    for image_id in image_id_dirs:
        # Construct path: e.g., .../A/A_processed_ramified/metrics_df_ramified.csv
        # Or .../iba1-1st_5/iba1-1st_5_processed_ramified_2d/metrics_df_ramified_2d.csv
        processed_folder_name = f"{image_id}_processed_{metrics_filename.replace('metrics_df_', '').replace('.csv', '')}"
        metrics_file_path = os.path.join(base_dir, image_id, processed_folder_name, metrics_filename)
        
        if os.path.exists(metrics_file_path):
            try:
                df = pd.read_csv(metrics_file_path)
                if df.empty:
                    # print(f"Warning: Metrics file for {image_id} is empty. Skipping.")
                    continue
                df['Image_ID_Full'] = image_id # Store the full original image ID
                df['Dataset_Tag'] = dataset_name_tag
                
                if derive_group_from_image_id_func:
                    df['Experimental_Group'] = df['Image_ID_Full'].apply(derive_group_from_image_id_func)
                
                all_metrics_data.append(df)
            except Exception as e:
                print(f"Error loading metrics for {image_id} from {metrics_file_path}: {e}. Skipping.")
        # else:
            # print(f"Warning: Metrics file not found for {image_id} at {metrics_file_path}")


    if not all_metrics_data:
        print(f"No metrics data loaded for {dataset_name_tag}.")
        return pd.DataFrame()

    dataset_df = pd.concat(all_metrics_data, ignore_index=True)
    
    # External Group Assignment (primarily for 3D)
    if group_assignment_df_external is not None:
        # The external group assignment uses a simplified Image_ID (e.g., "A")
        # We need a way to map Image_ID_Full to this simplified ID if necessary for merging
        # For 3D, Image_ID_Full is already the simple ID
        if 'Image_ID' not in dataset_df.columns: # If not already present from pattern matching
             dataset_df['Image_ID'] = dataset_df['Image_ID_Full'] # Simple case for 3D

        dataset_df = pd.merge(dataset_df, group_assignment_df_external, on='Image_ID', how='left', suffixes=('', '_external'))
        # If 'Experimental_Group_external' exists and 'Experimental_Group' was not derived, use it
        if 'Experimental_Group_external' in dataset_df.columns and 'Experimental_Group' not in dataset_df.columns:
            dataset_df['Experimental_Group'] = dataset_df['Experimental_Group_external']
        elif 'Experimental_Group_external' in dataset_df.columns and 'Experimental_Group' in dataset_df.columns:
            # Prefer externally assigned group if both exist and external is not NaN
            dataset_df['Experimental_Group'] = np.where(dataset_df['Experimental_Group_external'].notna(), 
                                                        dataset_df['Experimental_Group_external'], 
                                                        dataset_df['Experimental_Group'])
        if 'Experimental_Group_external' in dataset_df.columns:
            dataset_df.drop(columns=['Experimental_Group_external'], inplace=True)


    if 'Experimental_Group' not in dataset_df.columns:
        dataset_df['Experimental_Group'] = "DefaultGroup" # Fallback if no group info

    print(f"Finished loading {dataset_name_tag}: {len(dataset_df)} total cells from {dataset_df['Image_ID_Full'].nunique()} images.")
    print(f"Value counts for Experimental_Group in {dataset_name_tag}:\n{dataset_df['Experimental_Group'].value_counts(dropna=False)}")
    return dataset_df

# --- Define pattern matching functions for image directories ---
def is_3d_image_dir(dir_name, base_path): # base_path unused here but good for consistency
    return os.path.isdir(os.path.join(base_path, dir_name)) and len(dir_name) == 1 and dir_name.isalpha() and dir_name.isupper()

def is_2d_image_dir(dir_name, base_path):
    return os.path.isdir(os.path.join(base_path, dir_name)) and dir_name.startswith("iba1-")

# --- Define group derivation function for 2D ---
def derive_2d_group(image_id_full):
    match = re.search(r"iba1-(\d+)(?:st|nd|rd|th)", image_id_full) # e.g. iba1-1st, iba1-2nd
    if match:
        return f"2D_Batch{match.group(1)}"
    return "2D_Other" # Fallback

# --- Generic plotting function (plot_metric_boxplot from previous version) ---
# (This function is largely the same, ensure it uses passed `all_stats_records_list` and `export_dir`)
def plot_metric_boxplot(metric_col_name, data_df_source, dataset_tag_str, plot_title_suffix, 
                        all_stats_list_ref, export_directory_path, 
                        valid_groups_list, palette_map_orig, palette_map_pastel,
                        y_label_override=None):
    # Ensure this function is defined as in the previous response, with modifications for export_directory_path
    # and appending to all_stats_list_ref.
    # For brevity, I'm not repeating the full function here.
    # Key changes needed inside this function:
    # 1. Use `export_directory_path` for saving PDFs.
    # 2. Append to `all_stats_list_ref` instead of global `all_stats_records`.
    # 3. Take `valid_groups_list`, `palette_map_orig`, `palette_map_pastel` as arguments.

    print(f"--- ({dataset_tag_str}) Analyzing {format_axis_label(metric_col_name)} ---")
    if metric_col_name not in data_df_source.columns or not valid_groups_list:
        # print(f"Skipping {format_axis_label(metric_col_name)} for {dataset_tag_str}: column missing or no valid groups.")
        return

    # Histogram
    fig_hist, ax_hist = plt.subplots(figsize=(12, 7))
    sns.histplot(data=data_df_source.dropna(subset=['Experimental_Group']), x=metric_col_name, hue='Experimental_Group', kde=True, multiple='stack', palette=palette_map_orig, hue_order=valid_groups_list, ax=ax_hist)
    ax_hist.set_title(f'({dataset_tag_str}) Distribution of {format_axis_label(metric_col_name)}')
    ax_hist.set_xlabel(format_axis_label(metric_col_name)); ax_hist.set_ylabel('Cell Count')
    if valid_groups_list: ax_hist.legend(title='Exp. Group', handles=[plt.Rectangle((0,0),1,1, color=palette_map_orig[g]) for g in valid_groups_list], labels=valid_groups_list)
    plt.tight_layout()
    pdf_path_hist = os.path.join(export_directory_path, f'histogram_{dataset_tag_str}_{metric_col_name}.pdf')
    plt.savefig(pdf_path_hist, format='pdf', bbox_inches='tight'); plt.close(fig_hist)
    # print(f"Saved: {pdf_path_hist}")

    # Boxplot
    agg_data = data_df_source.groupby(['Image_ID_Full', 'Experimental_Group'])[metric_col_name].mean().reset_index()
    agg_data = agg_data.dropna(subset=['Experimental_Group', metric_col_name])
    n_counts = agg_data['Experimental_Group'].value_counts().reindex(valid_groups_list).fillna(0).astype(int)

    if not agg_data.empty:
        fig_box, ax_box = plt.subplots(figsize=(8, 6))
        sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)
        for g_name_plot in valid_groups_list:
            current_g_data_plot = agg_data[agg_data['Experimental_Group'] == g_name_plot]
            if not current_g_data_plot.empty:
                sns.stripplot(data=current_g_data_plot, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, color=palette_map_orig[g_name_plot], alpha=0.9, jitter=0.2, size=5, ax=ax_box)
        
        y_max_overall = agg_data[metric_col_name].max() if not agg_data[metric_col_name].empty else 0
        max_n_text_y = y_max_overall
        for i, g_name_text in enumerate(valid_groups_list):
            n_val = n_counts.get(g_name_text, 0)
            g_points = agg_data[agg_data['Experimental_Group'] == g_name_text][metric_col_name]
            text_y_val = y_max_overall * 1.02
            if not g_points.empty:
                q75 = g_points.quantile(0.75); iqr = q75 - g_points.quantile(0.25)
                upper_w = q75 + 1.5 * iqr if iqr > 0 else q75
                max_strip = g_points.max() if not g_points.empty else q75
                text_y_val = max(upper_w, max_strip) * 1.05
            ax_box.text(i, text_y_val, f"n={n_val}", ha='center', va='bottom', fontsize=9)
            max_n_text_y = max(max_n_text_y, text_y_val)

        ax_box.set_title(f'({dataset_tag_str}) Avg {format_axis_label(metric_col_name)} {plot_title_suffix}')
        ax_box.set_xlabel('Exp. Group'); ax_box.set_ylabel(y_label_override if y_label_override else f'Avg {format_axis_label(metric_col_name)}')
        
        p_val = None; y_lim_top = max_n_text_y * 1.05
        if len(valid_groups_list) == 2:
            g1n, g2n = valid_groups_list[0], valid_groups_list[1]
            g1d = agg_data[agg_data['Experimental_Group'] == g1n][metric_col_name].dropna()
            g2d = agg_data[agg_data['Experimental_Group'] == g2n][metric_col_name].dropna()
            if len(g1d) >= 2 and len(g2d) >= 2:
                s_stat, p_val = ttest_ind(g1d, g2d, nan_policy='omit')
                # print(f"T-test ({dataset_tag_str} Avg {format_axis_label(metric_col_name)}) {g1n} vs {g2n}: S={s_stat:.3f}, P={p_val:.4f}")
                all_stats_list_ref.append({'Dataset':dataset_tag_str, 'Metric': f"Avg {format_axis_label(metric_col_name)}", 'Comparison': f"{g1n} vs {g2n}", 'Test': 'T-test', 'Statistic': s_stat, 'P_Value': p_val, 'Significance': get_significance_asterisks(p_val)})
                y_bar = max_n_text_y * 1.08; y_bar = max(y_bar, max(g1d.max() if not g1d.empty else 0, g2d.max() if not g2d.empty else 0) * 1.15)
                t_level = y_bar * 1.05; tick_h = (y_bar - max_n_text_y) * 0.1 if (y_bar - max_n_text_y) > 0 else y_bar * 0.02
                ax_box.plot([0,1],[y_bar,y_bar],lw=1.5,c='k'); ax_box.plot([0,0],[y_bar-tick_h,y_bar],lw=1.5,c='k'); ax_box.plot([1,1],[y_bar-tick_h,y_bar],lw=1.5,c='k')
                ax_box.text(0.5, t_level, format_p_value_for_display(p_val), ha='center', va='bottom', fontsize=10)
                y_lim_top = max(y_lim_top, t_level*1.05)
        elif len(valid_groups_list) > 2 : # ANOVA
            samples_anova = [d[metric_col_name].dropna() for _, d in agg_data.groupby('Experimental_Group')]
            valid_samples_anova = [s for s in samples_anova if len(s) >=2]
            if len(valid_samples_anova) == len(agg_data['Experimental_Group'].unique()):
                f_stat, p_val = f_oneway(*valid_samples_anova)
                all_stats_list_ref.append({'Dataset':dataset_tag_str, 'Metric': f"Avg {format_axis_label(metric_col_name)}", 'Comparison': f"Across {len(valid_groups_list)} groups", 'Test': 'ANOVA', 'Statistic': f_stat, 'P_Value': p_val, 'Significance': get_significance_asterisks(p_val)})

        curr_ylim_b, _ = ax_box.get_ylim(); ax_box.set_ylim(bottom=curr_ylim_b, top=y_lim_top)
        plt.tight_layout()
        pdf_path_box = os.path.join(export_directory_path, f'boxplot_{dataset_tag_str}_{metric_col_name}.pdf')
        plt.savefig(pdf_path_box, format='pdf', bbox_inches='tight'); plt.close(fig_box)
        # print(f"Saved: {pdf_path_box}")
    # print("-" * 30 + "\n") #Less verbose

# --- Main Analysis ---
# 1. Load 3D Data
group_df_3d = None
try:
    group_df_raw_3d = pd.read_csv(GROUP_INFO_FILE_3D)
    group_df_raw_3d = group_df_raw_3d.rename(columns={'Unnamed: 0': 'Batch'})
    group_assignments_3d = []
    for _, row in group_df_raw_3d.iterrows():
        if pd.notna(row['Group1']): group_assignments_3d.append({'Image_ID': str(row['Group1']).strip(), 'Experimental_Group': '3D_Group1'}) # Prefix group names
        if pd.notna(row['Group2']): group_assignments_3d.append({'Image_ID': str(row['Group2']).strip(), 'Experimental_Group': '3D_Group2'})
    group_df_3d = pd.DataFrame(group_assignments_3d)
except Exception as e:
    print(f"Could not load 3D group assignments: {e}")

data_3d_df = load_and_process_dataset("3D", BASE_DIR_3D, METRICS_FILENAME_3D, 
                                      is_3d_image_dir, group_assignment_df_external=group_df_3d)

# 2. Analyze 3D Data (if loaded)
if not data_3d_df.empty:
    export_dir_3d = os.path.join(EXPORT_BASE_DIRECTORY, "3D_Analysis")
    if not os.path.exists(export_dir_3d): os.makedirs(export_dir_3d)
    
    valid_groups_3d = sorted(data_3d_df['Experimental_Group'].dropna().unique())
    palette_map_orig_3d = {g: ORIGINAL_CUSTOM_PALETTE[i % len(ORIGINAL_CUSTOM_PALETTE)] for i, g in enumerate(valid_groups_3d)}
    palette_map_pastel_3d = {g: PASTEL_PALETTE[i % len(PASTEL_PALETTE)] for i, g in enumerate(valid_groups_3d)}

    # Cell counts for 3D
    # (Simplified cell count plot, full logic is in plot_metric_boxplot if adapted)
    cells_3d_df = data_3d_df.groupby(['Image_ID_Full', 'Experimental_Group']).size().reset_index(name='num_cells')
    plot_metric_boxplot('num_cells', cells_3d_df.rename(columns={'num_cells':'num_cells_3d_placeholder'}), "3D", "per Image (Cell Count)", all_stats_records, export_dir_3d, valid_groups_3d, palette_map_orig_3d, palette_map_pastel_3d, y_label_override="Number of Cells")


    metrics_3d = ['shortest_distance_um', 'skan_num_branches', 'skan_total_length_um', 
                  'skan_avg_branch_length_um', 'skan_num_junctions', 'skan_num_endpoints', 
                  'sphericity', 'volume_um3']
    for metric in metrics_3d:
        plot_metric_boxplot(metric, data_3d_df, "3D", "per Image", all_stats_records, export_dir_3d, valid_groups_3d, palette_map_orig_3d, palette_map_pastel_3d)

# 3. Load 2D Data
data_2d_df = load_and_process_dataset("2D", BASE_DIR_2D, METRICS_FILENAME_2D, 
                                      is_2d_image_dir, 
                                      derive_group_from_image_id_func=derive_2d_group)

# 4. Analyze 2D Data (if loaded)
if not data_2d_df.empty:
    export_dir_2d = os.path.join(EXPORT_BASE_DIRECTORY, "2D_Analysis")
    if not os.path.exists(export_dir_2d): os.makedirs(export_dir_2d)

    valid_groups_2d = sorted(data_2d_df['Experimental_Group'].dropna().unique())
    palette_map_orig_2d = {g: ORIGINAL_CUSTOM_PALETTE[i % len(ORIGINAL_CUSTOM_PALETTE)] for i, g in enumerate(valid_groups_2d)} # Use different colors if desired
    palette_map_pastel_2d = {g: PASTEL_PALETTE[i % len(PASTEL_PALETTE)] for i, g in enumerate(valid_groups_2d)}
    
    # Cell counts for 2D
    cells_2d_df = data_2d_df.groupby(['Image_ID_Full', 'Experimental_Group']).size().reset_index(name='num_cells')
    plot_metric_boxplot('num_cells', cells_2d_df.rename(columns={'num_cells':'num_cells_2d_placeholder'}), "2D", "per Image (Cell Count)", all_stats_records, export_dir_2d, valid_groups_2d, palette_map_orig_2d, palette_map_pastel_2d, y_label_override="Number of Cells")

    metrics_2d = ['shortest_distance_um', 'area_um2', 'perimeter_um', 'circularity', 
                  'eccentricity', 'solidity', 'major_axis_length_um', 'minor_axis_length_um',
                  'skan_num_branches', 'skan_total_length_um', 'skan_avg_branch_length_um',
                  'skan_num_junctions', 'skan_num_endpoints']
    for metric in metrics_2d:
        if metric in data_2d_df.columns:
            plot_metric_boxplot(metric, data_2d_df, "2D", "per Image", all_stats_records, export_dir_2d, valid_groups_2d, palette_map_orig_2d, palette_map_pastel_2d)
        # else:
            # print(f"Metric {metric} not found in 2D data, skipping.")


# 5. Specific 3D Group1 vs. All 2D shortest_distance_um comparison
print("\n--- Comparing 3D Group1 vs ALL 2D (Shortest Distance) ---")
if not data_3d_df.empty and '3D_Group1' in data_3d_df['Experimental_Group'].unique() and not data_2d_df.empty:
    # Aggregate 3D Group1 data
    data_3d_g1_dist_agg = data_3d_df[data_3d_df['Experimental_Group'] == '3D_Group1'].groupby('Image_ID_Full')['shortest_distance_um'].mean().reset_index()
    data_3d_g1_dist_agg['Comparison_Group'] = '3D_Group1'
    
    # Aggregate all 2D data (it might have multiple '2D_BatchX' groups, so combine them under one label for this specific comparison)
    data_2d_dist_agg = data_2d_df.groupby('Image_ID_Full')['shortest_distance_um'].mean().reset_index()
    data_2d_dist_agg['Comparison_Group'] = '2D_All'

    comparison_df = pd.concat([data_3d_g1_dist_agg, data_2d_dist_agg], ignore_index=True)
    comparison_valid_groups = ['3D_Group1', '2D_All']
    
    # Define palettes for this specific comparison
    # Use first two colors from original palette for consistency, or define new ones
    comp_palette_map_orig = {comparison_valid_groups[0]: ORIGINAL_CUSTOM_PALETTE[0], comparison_valid_groups[1]: ORIGINAL_CUSTOM_PALETTE[1]}
    comp_palette_map_pastel = {comparison_valid_groups[0]: PASTEL_PALETTE[0], comparison_valid_groups[1]: PASTEL_PALETTE[1]}


    export_dir_comparison = os.path.join(EXPORT_BASE_DIRECTORY, "3D_vs_2D_Comparison")
    if not os.path.exists(export_dir_comparison): os.makedirs(export_dir_comparison)

    # Use the generic plot function for this comparison
    # We pass 'comparison_df' which has 'shortest_distance_um' and 'Comparison_Group'
    plot_metric_boxplot(
        metric_col_name='shortest_distance_um',
        data_df_source=comparison_df.rename(columns={'Comparison_Group':'Experimental_Group'}), # Generic func expects 'Experimental_Group'
        dataset_tag_str="3D-G1_vs_2D", # Tag for filenames
        plot_title_suffix="per Image",
        all_stats_list_ref=all_stats_records,
        export_directory_path=export_dir_comparison,
        valid_groups_list=comparison_valid_groups,
        palette_map_orig=comp_palette_map_orig,
        palette_map_pastel=comp_palette_map_pastel,
        y_label_override=f"Avg {format_axis_label('shortest_distance_um')}"
    )
else:
    print("Skipping 3D Group1 vs 2D comparison: 3D_Group1 not found in 3D data, or 2D data is empty.")


# --- Export All Statistics ---
if all_stats_records:
    stats_df = pd.DataFrame(all_stats_records)
    stats_csv_path = os.path.join(EXPORT_BASE_DIRECTORY, 'summary_statistics_combined.csv')
    stats_df.to_csv(stats_csv_path, index=False)
    print(f"\n--- Combined summary statistics exported to: {stats_csv_path} ---")
    # print(stats_df) # Optionally print
else:
    print("\n--- No statistical tests were performed or recorded. ---")

print("\n--- Main Analysis and Export Complete ---")

Created base export directory: /home/kirill/Desktop/For_Kirill/Takeshi_analysis_combined

--- Loading Dataset: 3D ---
Found 14 potential image directories for 3D.
Finished loading 3D: 2108 total cells from 14 images.
Value counts for Experimental_Group in 3D:
Experimental_Group
3D_Group1    1064
3D_Group2    1044
Name: count, dtype: int64
--- (3D) Analyzing Num Cells ---
--- (3D) Analyzing Shortest Distance µm ---


/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)


--- (3D) Analyzing Skan Num Branches ---


/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)


--- (3D) Analyzing Skan Total Length µm ---


...

/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)
/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)


--- (2D) Analyzing Skan Avg Branch Length µm ---
--- (2D) Analyzing Skan Num Junctions ---


/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)
/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)


--- (2D) Analyzing Skan Num Endpoints ---

--- Comparing 3D Group1 vs ALL 2D (Shortest Distance) ---
--- (3D-G1_vs_2D) Analyzing Shortest Distance µm ---

--- Combined summary statistics exported to: /home/kirill/Desktop/For_Kirill/Takeshi_analysis_combined/summary_statistics_combined.csv ---

--- Main Analysis and Export Complete ---


/tmp/ipykernel_13648/818978594.py:201: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(data=agg_data, x='Experimental_Group', y=metric_col_name, order=valid_groups_list, palette=palette_map_pastel, showfliers=False, width=BOXPLOT_WIDTH, linewidth=BOXPLOT_LINEWIDTH, ax=ax_box)
